In [1]:
import pandas as pd
import numpy as np
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import StandardScaler

In [2]:
# Load your final inventory DataFrame
inventory_df = pd.read_csv("inventory_final.csv", parse_dates = ["Date"])
all_rows = []

In [3]:
for pid in inventory_df["Product ID"].unique():
    df = inventory_df[inventory_df["Product ID"] == pid].copy()
    df = df.sort_values("Date").set_index("Date")

    future_dates = pd.date_range(df.index.max() + pd.offsets.MonthBegin(1),
                                 periods=6, freq="MS")

    # --- INVENTORY FORECAST ---
    y_inv = df["Inventory Level"]
    X_inv = df[["Total Units Sold", "Total Sales", "Total Profit", "Units Ordered"]]

    last_inv = y_inv.iloc[-1]
    try:
        auto_m = auto_arima(
            y_inv, exogenous=X_inv, seasonal=True, m=12,
            stepwise=True, suppress_warnings=True, error_action="ignore"
        )
        order, seasonal_order = auto_m.order, auto_m.seasonal_order

        y_scaler = StandardScaler()
        X_scaler = StandardScaler()
        y_inv_s = y_scaler.fit_transform(y_inv.values.reshape(-1,1)).flatten()
        X_inv_s = X_scaler.fit_transform(X_inv)

        model = SARIMAX(
            y_inv_s, exog=X_inv_s,
            order=order, seasonal_order=seasonal_order,
            enforce_stationarity=False, enforce_invertibility=False
        )
        res = model.fit(disp=False)

        last_exog = X_inv.iloc[[-1]]
        X_inv_fut = pd.DataFrame(
            np.repeat(last_exog.values, 6, axis=0),
            columns=last_exog.columns, index=future_dates
        )
        X_inv_fut_s = X_scaler.transform(X_inv_fut)
        inv_pred_s = res.predict(start=len(df), end=len(df)+5, exog=X_inv_fut_s)
        inv_pred = y_scaler.inverse_transform(inv_pred_s.reshape(-1,1)).flatten()
    except Exception:
        inv_pred = np.full(6, last_inv)

    # --- SALES FORECAST ---
    # --- SALES FORECAST: univariate SARIMA on Total Units Sold ---
    y_sal = df["Total Sales"]

    last_sal = y_sal.iloc[-1]
    try:
        # auto ARIMA on sales only
        auto_m2 = auto_arima(
            y_sal,
            seasonal=True,
            m=12,
            stepwise=True,
            suppress_warnings=True,
            error_action="ignore"
        )
        order2, seasonal_order2 = auto_m2.order, auto_m2.seasonal_order

        # scale target
        y_scaler2 = StandardScaler()
        y_sal_s = y_scaler2.fit_transform(y_sal.values.reshape(-1,1)).flatten()

        # fit SARIMA (no exog)
        model2 = SARIMAX(
            y_sal_s,
            order=order2,
            seasonal_order=seasonal_order2,
            enforce_stationarity=False,
            enforce_invertibility=False
        )
        res2 = model2.fit(disp=False)

        # forecast next 6
        sal_pred_s = res2.predict(
            start=len(df),
            end=len(df)+5
        )
        sal_pred = y_scaler2.inverse_transform(sal_pred_s.reshape(-1,1)).flatten()
    except Exception:
        sal_pred = np.full(6, last_sal)

    for date, inv_f, sal_f in zip(future_dates, inv_pred, sal_pred):
        all_rows.append({
            "Product ID": pid,
            "Date": date,
            "Inventory Forecast": inv_f,
            "Sales Forecast": sal_f
        })


C:\Users\neel9\anaconda3\envs\yolov-env\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\neel9\anaconda3\envs\yolov-env\lib\site-packages\pmdarima\arima\auto.py:444: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '
C:\Users\neel9\anaconda3\envs\yolov-env\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\neel9\anaconda3\envs\yolov-env\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\neel9\anaconda3\envs\yolov-env\lib\site-packages\pmdarima\arima\auto.py:444: UserWarning: Input time-series is completely 

In [4]:
# Build DataFrame and save
forecast_df = pd.DataFrame(all_rows)
forecast_df.to_csv("all_forecasts.csv", index = False)

In [5]:
forecast_df

,Product ID,Date,Inventory Forecast,Sales Forecast
0,19,2017-05-01,3.000000,249.979996
1,19,2017-06-01,3.000000,249.979996
2,19,2017-07-01,3.000000,249.979996
3,19,2017-08-01,3.000000,249.979996
4,19,2017-09-01,3.000000,249.979996
...,...,...,...,...
517,1073,2017-06-01,3027.518091,96195.192646
518,1073,2017-07-01,3076.348197,96195.192646
519,1073,2017-08-01,3109.896848,96195.192646
520,1073,2017-09-01,3132.946398,96195.192646
